In [ ]:
import xarray as xr
import numpy as np
import rioxarray
import os
from rasterio.enums import Resampling

# 1. Use the downloaded ETOPO1 file
etopo_file = "etopo180_8571_f367_229e.nc"

if not os.path.exists(etopo_file):
    raise FileNotFoundError(f"ETOPO1 file not found: {etopo_file}")

print(f"Using ETOPO1 file: {etopo_file}")

# 2. Open and prepare dataset
ds = xr.open_dataset(etopo_file, engine="netcdf4")
print(f"Dataset variables: {list(ds.data_vars)}")
print(f"Dataset coordinates: {list(ds.coords)}")

if "altitude" in ds.data_vars:
    elev = ds["altitude"]
elif "z" in ds.data_vars:
    elev = ds["z"]
else:
    elev = ds[list(ds.data_vars)[0]]

# Check current coordinate names
print(f"Elevation variable coordinates: {list(elev.coords)}")

# Rename to standard names if needed
coord_mapping = {}
if "latitude" in elev.coords:
    coord_mapping["latitude"] = "lat"
if "longitude" in elev.coords:
    coord_mapping["longitude"] = "lon"

if coord_mapping:
    elev = elev.rename(coord_mapping)

# Set CRS and spatial dimensions
elev = elev.rio.write_crs("EPSG:4326")
elev = elev.rio.set_spatial_dims(x_dim="lon", y_dim="lat")

# 3. Define target NASA POWER / MERRA-2 grid
lats = np.arange(-90 + 0.25, 90, 0.1)
lons = np.arange(-180 + 0.3125, 180, 0.1)
template_ds = xr.Dataset(coords={"lat": lats, "lon": lons})
template_ds = template_ds.rio.write_crs("EPSG:4326")
template_ds = template_ds.rio.set_spatial_dims(x_dim="lon", y_dim="lat")

# 4. Reproject / resample with mean aggregation
print("Reprojecting / averaging to MERRA-2 grid...")
elev_coarse = elev.rio.reproject_match(template_ds, resampling=Resampling.average)

# 5. Build final dataset & save WELEV
welev = elev_coarse
welev.attrs["units"] = "m"
welev.attrs["long_name"] = "Mean elevation (m)"
welev.attrs["description"] = "From ETOPO1 (ice surface) averaged to 0.1° × 0.1° grid via ERDDAP source"

ds_out = xr.Dataset({"WELEV": welev})
ds_out.attrs.update({
    "title": "Global WELEV (mean elevation) on NASA POWER / MERRA-2 grid",
    "source": "ETOPO1 via NOAA ERDDAP",
    "grid_resolution": "0.1° × 0.1°"
})

out_file = "welev_merra2_grid_1.nc"
ds_out.to_netcdf(out_file)
print(f"Saved WELEV file: {out_file}")

✅ Using ETOPO1 file: etopo180_8571_f367_229e.nc
Dataset variables: ['altitude']
Dataset coordinates: ['latitude', 'longitude']
Elevation variable coordinates: ['latitude', 'longitude']
🔁 Reprojecting / averaging to MERRA-2 grid...
✅ Saved WELEV file: welev_merra2_grid_1.nc
✅ Saved WELEV file: welev_merra2_grid_1.nc
